## Function as first class citizen

In python function are first class citizen -  You can assign them to variables, store them in data structures, pass them as arguments to other functions, and even return them as values from other functions.

In [19]:
def square(x):
    return x*x

#f = square(5)   
#print(square)
#print(f)

f = square         # assigned function to a variable
print(f)
print(f(5))

<function square at 0x7f15fce3d1e0>
25


In [ ]:
def my_map(func, arg_list): # taking function as argument
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

squares = my_map(square, [1,2,3,4,5])
squares

In [20]:
def sort_by_last_letter(strings):
    def last_letter(s):                       # Local function
        return s[-1]
    #print(last_letter)                       # uncomment and call multiple times, see the address of function.
    return sorted(strings, key=last_letter)   # returning a function

sort_by_last_letter(['hello', 'from', 'a', 'local', 'function'])

['a', 'local', 'from', 'function', 'hello']

## Variables Scope

*  **LEGB** rule for name lookup: first the Local scope is checked, then any Enclosing scope, next the
   Global scope, and finally the Builtin scope

In [21]:
count = 0

def show_count():
    print(count)
    
def set_count(c):
    count = c

show_count()
set_count(5)
show_count()

0
0


### The global  and nonlocal keyword

In [22]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        message = 'local'
    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: enclosing
global message: global


In [23]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        global message                    
        message = 'local'
    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: enclosing
global message: local


In [24]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        nonlocal message
        message = 'local'
    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: local
global message: global


## Closure

That is, once a local function is returned from its enclosing scope, that enclosing scope is gone, along with any local objects it defined. How can the local function operate without that enclosing scope?

The answer is that the local function forms what is known as a closure. A closure essentially
remembers the objects from the enclosing scope that the local function needs. It then keeps
them alive so that when the local function is executed they can still be used.

If a function closes over any objects, then that function has a \_\_closure\_\_ attribute which maintains
the necessary references to those objects. We can see that in a simple example:

In [26]:
def enclosing():
    x = 'closed over'
    def local_func():
        print(x)
    return local_func

lf = enclosing()
lf()
lf.__closure__

closed over


(<cell at 0x7f15fc573468: str object at 0x7f15fc57e530>,)

## Function factories

These factories are functions that
return other functions, where the returned functions are specialized in some way based on
arguments to the factory

In [27]:
def raise_to(exp):               # Function Factory
    def raise_to_exp(x):
        return pow(x, exp)
    return raise_to_exp

square = raise_to(2) # exp = 2
square(7) # x = 7   7^2

49

In [30]:
# Another practical example of closure
import time

def make_timer():
    last_called = None # Never
    
    def elapsed():
        nonlocal last_called
        now = time.time()
        if last_called is None:
            last_called = now
            return None
        result = now - last_called
        last_called = now
        return result
    
    return elapsed

In [31]:
t = make_timer()
print(t())

None


In [35]:
t()

1.711867094039917

## Decorator

In Python, a decorator is a callable object that takes in a callable and returns a callable.
At a high level, decorators are a way to modify or enhance existing functions in a non-intrusive
and maintainable way.

### Functions as callable objects

In [37]:
def sum(a, b):
    return a + b

a = sum
#dir(a)        # Find __call__ method

In [38]:
def my_decorator(func):
    def wrapper():
        print("something is happening bfore function is called")
        func()
        print("something is happening after the function is called")
    return wrapper

def say_hi():
    print("Hi MITRC")

say_hi = my_decorator(say_hi)
say_hi()

something is happening bfore function is called
Hi MITRC
something is happening after the function is called


In [ ]:
@my_decorator   # Syntactic sugar !  @ syntax for decorator
def say_hi():
    print("Hello MITRC")

say_hi()

In [40]:
def exception_handler(original_fn):
    def decorator_fn(*args, **kwargs):
        try:
            return original_fn(*args, **kwargs)
        except Exception as err:
            print(err)
    return decorator_fn
 
@exception_handler
def add(x, y):
    sum = x + y
    print(sum)
    return sum

add(5)

add() missing 1 required positional argument: 'y'


### Classes as decorators

In [41]:
class CallCount:
    def __init__(self, f):
        self.f = f
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.f(*args, **kwargs)
    
@CallCount
def hello(name):
    print('Hello, {}!'.format(name))


hello('Fred')
hello('Fred')
hello('Fred')
hello.count

Hello, Fred!
Hello, Fred!
Hello, Fred!


3

### Instances as decorators

In [ ]:
class Trace:
    def __init__(self):
        self.enabled = True
    
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
        return wrap
    
tracer = Trace()

@tracer
def rotate_list(l):
    return l[1:] + [l[0]]

l = [1, 2, 3]
l = rotate_list(l)
l

### Multiple decorators

In [42]:
def add_decorator_one(original_fn):
    def decorator_fn(*args, **kwargs):
        original_fn(*args, **kwargs)
        print("Hello from one")
    return decorator_fn
 
def add_decorator_two(original_fn):
    def decorator_fn(*args, **kwargs):
        original_fn(*args, **kwargs)
        print("Hello from two")
    return decorator_fn
  
@add_decorator_one
@add_decorator_two
def add(x, y):
    print(x + y)

add(4,5)

9
Hello from two
Hello from one


### Decorating methods

In [ ]:
class IslandMaker:
    def __init__(self, suffix):
        self.suffix = suffix
    
    @tracer
    def make_island(self, name):
        return name + self.suffix

### Decorators and function metadata

In [ ]:
def hello():
    "Print a well-known message."
    print('Hello, world!')
    
hello.__name__

In [ ]:
hello.__doc__

In [ ]:
def noop(f):
    def noop_wrapper():
        return f()
    return noop_wrapper

@noop
def hello():
    "Print a well-known message."
    print('Hello, world!')

hello.__name__
#hello.__doc__

In [ ]:
### Manually updating the Metadata

def noop(f):
    def noop_wrapper():
        return f()
    noop_wrapper.__name__ = f.__name__
    noop_wrapper.__doc__ = f.__doc__
    return noop_wrapper

In [ ]:
# Updating decorator metadata with functools.wraps

import functools

def noop(f):
    @functools.wraps(f)
    def noop_wrapper():
        return f()
    return noop_wrapper

### Decorator Factory

A decorator factory is a
function that returns a decorator; the actual decorator is customized based on the arguments
to the factory.

In [ ]:
# A decorator factory: it returns decorators
def check_non_negative(index):
# This is the actual decorator
    def validator(f):
    # This is the wrapper function
        def wrap(*args):
            if args[index] < 0:
                raise ValueError('Argument {} must be non-negative.'.format(index))
            return f(*args)
        return wrap
    return validator

@check_non_negative(1)
def create_list(value, size):
    return [value] * size

In [ ]:
create_list('a', 3)

In [ ]:
create_list(123, -6)